In [ ]:
import requests
import urllib.request
import os
from bs4 import BeautifulSoup

In [ ]:
def upravMeno(name):
    name = name.split(" ")
    s = ""
    for i in name:
        if i[-1] != ".":
            s += i.strip(",")+" "
    s = s.strip()
    s = s.replace("á", "a")
    s = s.replace("é", "e")
    s = s.replace("í", "i")
    s = s.replace("ó", "o")
    s = s.replace("ú", "u")
    s = s.replace("ý", "y")
    s = s.replace("ž", "z")
    s = s.replace("ť", "t")
    s = s.replace("č", "c")
    s = s.replace("š", "s")
    s = s.replace("ď", "d")
    s = s.replace("ň", "n")
    s = s.replace("ľ", "l")
    s = s.replace("ř", "r")
    s = s.replace("Ž", "Z")
    s = s.replace("Ť", "T")
    s = s.replace("Č", "C")
    s = s.replace("Š", "S")
    s = s.replace("Ď", "D")
    s = s.replace("Ň", "N")
    s = s.replace("Ľ", "L")
    s = s.replace("ô", "o")
    s = s.replace("ä", "a")
    s = s.replace(" ", "_")
    return s
    

### Ziskavanie fotiek

In [ ]:
basePhoto = "https://sluzby.fmph.uniba.sk/"
noPhotoFile = open("noPhoto.jpg", "rb")
noPhoto = noPhotoFile.read()
noPhotoFile.close()
    
def getImages(href, meno, kat, main):
    personPage = requests.get(href).text
    personParsed = BeautifulSoup(personPage, "html.parser")
    
    temp = personParsed.select("aside.span3 img")
    #print(temp)
    if temp != []:
        for i in temp:
            if i["alt"] != "photo":
                img = i["src"]
                imgData = requests.get(basePhoto+img)
                if imgData.content != noPhoto:
                    fileName = upravMeno(meno)
                    f = open(main+"/"+kat+"_"+fileName+".jpg", "wb")
                    f.write(imgData.content)
                    f.close()
    else:
        print("Pri", meno, "sa vyskytol problem s obrazkami")

        
def findPhotos(baseUrl, kat, main):    
    basePage = requests.get(baseUrl).text
    baseParsed = BeautifulSoup(basePage, "html.parser")

    links = baseParsed.select("table a")
    for i, clovek in enumerate(links):
        href = clovek["href"]
        if "javascript" not in href:
            meno = clovek.text
            #print(href, meno, upravMeno(meno))
            getImages(href, meno, kat, main) 
    print("Hotovo", kat)

In [ ]:
mainFolder = "photos"

try:
    os.mkdir(mainFolder)
except:
    print("Problem, asi uz existujuca zlozka")
             
katedry = {"KAG": "https://fmph.uniba.sk/pracoviska/katedra-algebry-a-geometrie/",
           "KAMS": "https://fmph.uniba.sk/pracoviska/katedra-aplikovanej-matematiky-a-statistiky/",
           "KMANM": "https://fmph.uniba.sk/pracoviska/katedra-matematickej-analyzy-a-numerickej-matematiky/",
           "KAFZM": "https://fmph.uniba.sk/pracoviska/katedra-astronomie-fyziky-zeme-a-meteorologie/", 
           "KEF": "https://fmph.uniba.sk/pracoviska/katedra-experimentalnej-fyziky/",
           "KJFB": "https://fmph.uniba.sk/pracoviska/katedra-jadrovej-fyziky-a-biofyziky/",
           "KTF": "https://fmph.uniba.sk/pracoviska/katedra-teoretickej-fyziky/",
           "AIN": "https://fmph.uniba.sk/pracoviska/katedra-aplikovanej-informatiky/",
           "INF": "https://fmph.uniba.sk/pracoviska/katedra-informatiky/",
           "KDMFI": "https://fmph.uniba.sk/pracoviska/katedra-didaktiky-matematiky-fyziky-a-informatiky/",
           "KJP": "https://fmph.uniba.sk/pracoviska/katedra-jazykovej-pripravy/",
           "KTVS": "https://fmph.uniba.sk/pracoviska/katedra-telesnej-vychovy-a-sportu/",
           "HIP_dek": "https://fmph.uniba.sk/pracoviska/dekanat/",
           "HIP_cpp": "https://fmph.uniba.sk/pracoviska/centrum-projektovej-podpory/",
           "HIP_kec": "https://fmph.uniba.sk/pracoviska/kniznicne-a-edicne-centrum/",
           "HIP_vc": "https://fmph.uniba.sk/pracoviska/vypoctove-centrum/",
           "HIP_vl": "https://fmph.uniba.sk/pracoviska/vyvojove-laboratorium/",
           "HIP_sb": "https://fmph.uniba.sk/pracoviska/sprava-budov/"}

for k, url in katedry.items():
    findPhotos(url, k, mainFolder)

#### Katedra aplikovanej informatiky naviac

In [ ]:
baseUrl = "https://dai.fmph.uniba.sk"
vsetci = "/w/People/sk"
kat = "AIN_extra"
noPhotoFile2 = open("noPhoto2.jpg", "rb")
noPhoto2 = noPhotoFile2.read()
noPhotoFile2.close()
    
def getImagesExtra(clovek, meno, kat, main):
    personPage = requests.get(baseUrl+clovek).text
    personParsed = BeautifulSoup(personPage, "html.parser")
    
    temp = personParsed.select("div#mw-content-text table a.image")
    #print(temp)
    if temp != []:
        for i in temp:
            if i["title"] != "@":
                personPage2 = requests.get(baseUrl+i["href"]).text
                personParsed2 = BeautifulSoup(personPage2, "html.parser")

                temp = personParsed2.select("div.fullImageLink img")
                #print(temp)
                if temp != []:
                    img = temp[0]["src"]
                    imgData = requests.get(baseUrl+img)
                    if imgData.content != noPhoto2:
                        fileName = upravMeno(meno)
                        f = open(main+"/"+kat+"_"+fileName+".jpg", "wb")
                        f.write(imgData.content)
                        f.close()
                else:
                    print("Pri", meno, "sa vyskytol problem s obrazkom 0")
    else:
        print("Pri", meno, "sa vyskytol problem s obrazkom")

basePage = requests.get(baseUrl+vsetci).text
baseParsed = BeautifulSoup(basePage, "html.parser")

links = baseParsed.select("table a")
for i, clovek in enumerate(links):
    href = clovek["href"]
    if href[:3] == "/w/":
        getImagesExtra(href, clovek["title"], kat, mainFolder)
        #print(href, clovek["title"])
        #print(href, clovek["title"], upravMeno(clovek["title"]))
        ...
        
print("Hotovo", kat)        
